In [2]:
!pip install qgrid
import qgrid
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler # Escalamiento estándar
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
import scipy as sc
import math
from numpy import random
from numpy import matlib
from scipy.spatial import distance
import matplotlib.pyplot as plt







/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
#Eliminamos las filas nulas
DatosConNulls = pd.read_csv("water_potability.csv", names = None, sep = ",")


In [4]:
DatosConNulls.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [5]:
#completar los valores en null con su media
df1=DatosConNulls[DatosConNulls['Potability']==1].copy()
df2=DatosConNulls[DatosConNulls['Potability']==0].copy()
df1['ph']=df1['ph'].replace(np.nan, df1['ph'].median())
df2['ph']=df2['ph'].replace(np.nan, df2['ph'].median())
df1['Sulfate']=df1['Sulfate'].replace(np.nan, df1['Sulfate'].median())
df2['Sulfate']=df2['Sulfate'].replace(np.nan, df2['Sulfate'].median())
df1['Trihalomethanes']=df1['Trihalomethanes'].replace(np.nan, df1['Trihalomethanes'].median())
df2['Trihalomethanes']=df2['Trihalomethanes'].replace(np.nan, df2['Trihalomethanes'].median())
Datos=pd.concat([df1,df2], axis=0, ignore_index=True)
Datos.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [6]:
X = Datos.iloc[:,0:9] #Caracteristicas
Nombres = Datos.columns
Y = Datos['Potability']#clases
#Ya que la diferencia entre las escalas de los datos de X, se hará una normalización de los datos 
scaler = StandardScaler()
scaler.fit(X)
X_escalado = scaler.transform(X)


In [7]:
def completarTablaRedesNeuronales (i,tablaRedesNeuronales,EV,IC,F1,ICF1,PR):
  tablaRedesNeuronales.loc[i,"Porcentaje de reduccion"] = str(PR)
  tablaRedesNeuronales.loc[i,"Eficiencia en validacion"] = str(EV) # reemplazar los valores
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaRedesNeuronales.loc[i,"f1-score"] = str(F1)
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaRedesNeuronales)

In [8]:
def completarTablaSoporteVectorial (i,tablaSoporteVectorial,EV,IC,PV,F1,ICF1,PR):
  tablaSoporteVectorial.loc[i,"Porcentaje de reduccion"] = str(PR)
  tablaSoporteVectorial.loc[i,"Eficiencia en validacion"] = str(EV)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaSoporteVectorial.loc[i, "% de Vectores de Soporte"] = str(PV)
  tablaSoporteVectorial.loc[i,"f1-score"] = str(F1)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaSoporteVectorial)

In [15]:
def completarTablaBoostingTree (pos,tablaBoostingTree,EV,IC,F1,ICF1,PR):
  tablaBoostingTree.loc[pos,"Porcentaje de reduccion"] = str(PR)
  tablaBoostingTree.loc[pos,"Eficiencia en validacion"] = str(EV) 
  tablaBoostingTree.loc[pos,"Intervalo de confianza"] = str(IC)
  tablaBoostingTree.loc[pos,"f1-score"] = str(F1)
  tablaBoostingTree.loc[pos,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaBoostingTree)

In [10]:
def select_features(modelo, n_features, fwd, fltg):
    """
    Feature Selection Function: helper para el uso de la libreria
    #Recibe 4 parámetros: 
    1. el modelo (clf para nuestro caso), 
    2. el número de características final que se quiere alcanzar
    3. Si es forward (True), si es Backward False, 
    4. Si es es flotante (True), sino False
    """
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=0,
           scoring='accuracy',
           cv=0, n_jobs=-1)
    
    return sfs

# 1. Redes neuronales

In [11]:
 #Integer encode
label_encoder = LabelEncoder()
integerY = label_encoder.fit_transform(Y)
print(integerY)

#Binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integerY = integerY.reshape(len(integerY), 1)
onehot_encoded = onehot_encoder.fit_transform(integerY)
print(onehot_encoded)

[1 1 1 ... 0 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import scipy as sc
import math
from numpy import random
from numpy import matlib


tablaRedesNeuronales= pd.DataFrame({
    'Ncaracteristicas' : pd.Series([3,5,7,8,9])})
Ncaracteristicas=[3,5,7,8,9]
Nfwd=[True,True,True,True,True]
Nfltg=[False,False,False,False,False]
Ncar = np.array([3,5,7,8,9])

for i in range (5):

  Folds = 4
  random.seed(19680801)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  PR=(Ncar[i]/9)*100
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      
      vector=(28,28)  

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
      mlp = MLPClassifier(hidden_layer_sizes=vector, activation='identity', max_iter=400,verbose=True)
      mlp.out_activation_='softmax'
      sf = select_features(mlp,Ncaracteristicas[i], Nfwd[i], Nfltg[i])

      sf=sf.fit(Xtrain,Ytrain)
      sf_xtrain = sf.transform(Xtrain)
      sf_xtest = sf.transform(Xtest)
      mlp=mlp.fit(sf_xtrain,Ytrain)

      Ytrain_pred = mlp.predict(sf_xtrain)
      Yest = mlp.predict(sf_xtest)

      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      j += 1
          
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaRedesNeuronales = completarTablaRedesNeuronales(i,tablaRedesNeuronales,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1),PR)

Iteration 1, loss = 0.71110596
Iteration 2, loss = 0.67041300
Iteration 3, loss = 0.66945392
Iteration 4, loss = 0.66869008
Iteration 5, loss = 0.66866200
Iteration 6, loss = 0.66925585
Iteration 7, loss = 0.66859751
Iteration 8, loss = 0.66793424
Iteration 9, loss = 0.66839164
Iteration 10, loss = 0.66828481
Iteration 11, loss = 0.66811238
Iteration 12, loss = 0.66871840
Iteration 13, loss = 0.66801126
Iteration 14, loss = 0.66846823
Iteration 15, loss = 0.66848362
Iteration 16, loss = 0.66877855
Iteration 17, loss = 0.66813755
Iteration 18, loss = 0.66836113
Iteration 19, loss = 0.66864351
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.68420153
Iteration 2, loss = 0.67094341
Iteration 3, loss = 0.66915606
Iteration 4, loss = 0.66922676
Iteration 5, loss = 0.66883853
Iteration 6, loss = 0.66922928
Iteration 7, loss = 0.66903907
Iteration 8, loss = 0.66939201
Iteration 9, loss = 0.66905134
Iteration 10, loss = 0.67030338


In [13]:
tablaRedesNeuronales

,Ncaracteristicas,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza de eficiencia,f1-score,Intervalo de confianza de f1-score
0,3,55.55555555555556,0.6098901098901099,0.0006105006105006638,0.4621012175471858,0.0007498897315181274
1,5,66.66666666666666,0.6098901098901099,0.0006105006105006638,0.4621012175471858,0.0007498897315181274
2,7,77.77777777777779,0.6114163614163615,0.002498581432195536,0.4676706812144208,0.007680354714662361
3,8,88.88888888888889,0.6105006105006104,0.002114836150877791,0.46564491487013304,0.005778225098358869
4,9,100.0,0.6120268620268621,0.0027809626309964735,0.46800597327908106,0.006723795777252328


# 2. Máquinas de soporte vectorial

In [11]:
import sklearn
from sklearn import preprocessing

#Se modifica cada etiqueta para que corresponda a un numero

dataSet=[]
label = np.unique(Y)
encoder=preprocessing.LabelEncoder()
encoder.fit(label)
data=encoder.transform(Y)
dataSet.append(data)
dataFrameY=pd.DataFrame(dataSet).values
dataFrameY=dataFrameY.T
dataFrameY = dataFrameY.astype(np.float)
dataFrameY=dataFrameY.ravel() 

In [19]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import qgrid

tablaSoporteVectorial = pd.DataFrame({
    'Ncaracteristicas' : pd.Series([3,5,7,8,9])})
Ncaracteristicas=[3,5,7,8,9]
Nfwd=[True,True,True,True,True]
Nfltg=[False,False,False,False,False]
Ncar = np.array([3,5,7,8,9])
for i in range (5):

  #Validamos el modelo
  Folds = 4
  random.seed(24524)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  f1 = np.zeros(Folds)
  j = 0
  porcentaje = np.zeros(4)
  PR=(Ncar[i]/9)*100

  for train, test in skf.split(X_escalado, dataFrameY):
      Xtrain = X_escalado[train,:]
      Ytrain = dataFrameY[train]
      Xtest = X_escalado[test,:]
      Ytest = dataFrameY[test]
      
      #Normalizamos los datos
      scaler = preprocessing.StandardScaler().fit(Xtrain)
      Xtrain = scaler.transform(Xtrain)
      Xtest = scaler.transform(Xtest)
      modelo = SVC(kernel= 'rbf', C=10, gamma=0.1,decision_function_shape='ovr') #Si es rbf
      
      sf = select_features(modelo,Ncaracteristicas[i], Nfwd[i], Nfltg[i])
      sf = sf.fit(Xtrain,Ytrain)
      sf_xtrain = sf.transform(Xtrain)
      sf_xtest = sf.transform(Xtest)
      modelo = modelo.fit(sf_xtrain,Ytrain)
      #Validación
      Ytrain_pred = modelo.predict(sf_xtrain)
      Yest = modelo.predict(sf_xtest)

      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      porcentaje[j] =  len(modelo.support_vectors_) / len(Xtrain)
      
      j += 1
          
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaSoporteVectorial=completarTablaSoporteVectorial(i,tablaSoporteVectorial,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(porcentaje),np.mean(f1),np.std(f1),PR)

Eficiencia durante el entrenamiento = 0.6799959299959301+-0.01445675242517047
Eficiencia durante la validación = 0.6355311355311355+-0.040130838655787066
f1-score  0.5840968295610975  +-  0.03667019049325399
Eficiencia durante el entrenamiento = 0.7287342287342288+-0.012013408337928709
Eficiencia durante la validación = 0.63003663003663+-0.04595411298304962
f1-score  0.5943333498899948  +-  0.04255257989408563
Eficiencia durante el entrenamiento = 0.7705535205535206+-0.010975341593938004
Eficiencia durante la validación = 0.6190476190476191+-0.04319480574190165
f1-score  0.5911278521481732  +-  0.04105939710635362
Eficiencia durante el entrenamiento = 0.7967032967032968+-0.007379513375852147
Eficiencia durante la validación = 0.6169108669108669+-0.03950250321285296
f1-score  0.5906230303587088  +-  0.04061641817486643
Eficiencia durante el entrenamiento = 0.8263125763125763+-0.008172376628797304
Eficiencia durante la validación = 0.605006105006105+-0.036898689221639976
f1-score  0.5859

In [21]:
tablaSoporteVectorial

,Ncaracteristicas,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza de eficiencia,% de Vectores de Soporte,f1-score,Intervalo de confianza de f1-score
0,3,55.55555555555556,0.6355311355311355,0.040130838655787066,0.7026862026862026,0.5840968295610975,0.03667019049325399
1,5,66.66666666666666,0.63003663003663,0.04595411298304962,0.6584249084249084,0.5943333498899948,0.04255257989408563
2,7,77.77777777777779,0.6190476190476191,0.04319480574190165,0.6712454212454213,0.5911278521481732,0.04105939710635362
3,8,88.88888888888889,0.6169108669108669,0.03950250321285296,0.6788766788766789,0.5906230303587088,0.04061641817486643
4,9,100.0,0.605006105006105,0.036898689221639976,0.6903744403744403,0.5859246133455577,0.035034982603876705


# 3. Gradient boosting tree

In [17]:
tablaBoostingTree =  pd.DataFrame({
    'Ncaracteristicas' : pd.Series([3,5,7,8,9])})
Ncaracteristicas=[3,5,7,8,9]
Nfwd=[True,True,True,True,True]
Nfltg=[False,False,False,False,False]
Ncar = np.array([3,5,7,8,9])



for i in range(5):
  clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.3	,max_depth=5, random_state=0)
  Folds = 4
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  PR=(Ncar[i]/9)*100
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      sf = select_features(clf,Ncaracteristicas[i], Nfwd[i], Nfltg[i])
      sf = sf.fit(Xtrain,Ytrain)
      sf_xtrain = sf.transform(Xtrain)
      sf_xtest = sf.transform(Xtest)
      clf.fit(sf_xtrain,Ytrain)
      #Validación
      Ytrain_pred = clf.predict(sf_xtrain)
      Yest = clf.predict(sf_xtest)

      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaBoostingTree=completarTablaBoostingTree(i,tablaBoostingTree,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1),PR)
  

Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.18894993894993894+-0.32727089434833184
f1-score  0.1885606038946987  +-  0.32659654625148804
Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.19230769230769232+-0.3330866937632457
f1-score  0.19141225927645192  +-  0.33153575825836185
Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.18833943833943834+-0.32621347627289293
f1-score  0.187127575732411  +-  0.32411446866572874
Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.18925518925518925+-0.3277996033860512
f1-score  0.1877274879491739  +-  0.32515354710524336
Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.19444444444444445+-0.3367876570272817
f1-score  0.19324473254885624  +-  0.3347096950696782


In [18]:
tablaBoostingTree

,Ncaracteristicas,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza,f1-score,Intervalo de confianza de f1-score
0,3,55.55555555555556,0.18894993894993894,0.32727089434833184,0.1885606038946987,0.32659654625148804
1,5,66.66666666666666,0.19230769230769232,0.3330866937632457,0.19141225927645192,0.33153575825836185
2,7,77.77777777777779,0.18833943833943834,0.32621347627289293,0.187127575732411,0.32411446866572874
3,8,88.88888888888889,0.18925518925518925,0.3277996033860512,0.1877274879491739,0.32515354710524336
4,9,100.0,0.19444444444444445,0.3367876570272817,0.19324473254885624,0.3347096950696782
